In [17]:
import pandas as pd
import numpy as np
import glob

In [18]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
csv_files = glob.glob('/content/drive/My Drive/Summer Research 24/NycMayorVote/NYCData*.csv')

df_list = [pd.read_csv(file) for file in csv_files]
data = pd.concat(df_list, ignore_index = True)
data = data.rename(columns={
'DEM Mayor Choice 1 of 5 Citywide (024306)': '1st',
'DEM Mayor Choice 2 of 5 Citywide (224306)': '2nd',
'DEM Mayor Choice 3 of 5 Citywide (324306)': '3rd',
'DEM Mayor Choice 4 of 5 Citywide (424306)': '4th',
'DEM Mayor Choice 5 of 5 Citywide (524306)': '5th'
})
data

,1st,2nd,3rd,4th,5th
0,Write-in,217796,221183,219978,217572
1,undervote,undervote,undervote,undervote,undervote
2,undervote,undervote,undervote,undervote,undervote
3,undervote,undervote,undervote,undervote,undervote
4,217605,219469,221183,221458,217572
...,...,...,...,...,...
1068467,217796,217572,218127,undervote,undervote
1068468,Write-in,undervote,undervote,undervote,undervote
1068469,217796,217572,undervote,undervote,undervote
1068470,217572,217796,218127,undervote,undervote


In [20]:
grouped_ballots = data.groupby(['1st', '2nd', '3rd', '4th', '5th']).size().reset_index(name='count')

grouped_ballots

,1st,2nd,3rd,4th,5th,count
0,217572,217572,217572,217572,217572,3432
1,217572,217572,217572,217572,217605,8
2,217572,217572,217572,217572,217654,1
3,217572,217572,217572,217572,217796,28
4,217572,217572,217572,217572,218127,14
...,...,...,...,...,...,...
77558,undervote,undervote,undervote,undervote,221183,9
77559,undervote,undervote,undervote,undervote,221458,5
77560,undervote,undervote,undervote,undervote,Write-in,3
77561,undervote,undervote,undervote,undervote,overvote,36


In [21]:

def process_votes(row):
    # Replace "undervote" and "overvote" with "Null"
    choices = ['Null' if x in ['undervote', 'overvote'] else x for x in row[:-1]]

    # Handle duplicates: keep only the first occurrence, rest should be "Null"
    unique_choices = []
    seen = set()
    for choice in choices:
        if choice not in seen and choice != 'Null':
            seen.add(choice)
            unique_choices.append(choice)
        else:
            unique_choices.append('Null')

    # Ensure there are exactly 5 choices by filling with "Null" if needed
    unique_choices = unique_choices[:5]  # In case we have more than 5 due to initial data structure
    if len(unique_choices) < 5:
        unique_choices += ['Null'] * (5 - len(unique_choices))

    # Rearrange the choices so that all valid choices are at the top
    valid_choices = [choice for choice in unique_choices if choice != 'Null']
    null_choices = ['Null'] * (5 - len(valid_choices))

    # Combine valid choices with null choices
    final_choices = valid_choices + null_choices

    # Return the new row with processed choices and original count
    return pd.Series(final_choices + [row['count']])

# Apply the function to each row
processed_df = grouped_ballots.apply(process_votes, axis=1)

# Assign the column names
processed_df.columns = ['1st', '2nd', '3rd', '4th', '5th', 'count']

print(processed_df)

            1st     2nd   3rd   4th   5th   count
0        217572    Null  Null  Null  Null    3432
1        217572  217605  Null  Null  Null       8
2        217572  217654  Null  Null  Null       1
3        217572  217796  Null  Null  Null      28
4        217572  218127  Null  Null  Null      14
...         ...     ...   ...   ...   ...     ...
77558    221183    Null  Null  Null  Null       9
77559    221458    Null  Null  Null  Null       5
77560  Write-in    Null  Null  Null  Null       3
77561      Null    Null  Null  Null  Null      36
77562      Null    Null  Null  Null  Null  120541

[77563 rows x 6 columns]


In [22]:
# Remove rows where all five vote columns are "Null"
processed_df = processed_df[~((processed_df['1st'] == 'Null') &
                             (processed_df['2nd'] == 'Null') &
                             (processed_df['3rd'] == 'Null') &
                             (processed_df['4th'] == 'Null') &
                             (processed_df['5th'] == 'Null'))]

# Reaggregate the DataFrame by summing the counts of identical voting patterns
aggregated_df = processed_df.groupby(['1st', '2nd', '3rd', '4th', '5th'], as_index=False)['count'].sum().sort_values(by='count', ascending=False).reset_index(drop=True)

print(aggregated_df)

          1st     2nd     3rd     4th       5th  count
0      217572    Null    Null    Null      Null  64274
1      217796    Null    Null    Null      Null  20566
2      219469    Null    Null    Null      Null  12771
3      219978    Null    Null    Null      Null  10258
4      217572  219469    Null    Null      Null   9833
...       ...     ...     ...     ...       ...    ...
62149  217796  218117  219469  217605    218922      1
62150  219469  218117  221183  217605    218922      1
62151  217796  218117  219469  217605      Null      1
62152  219469  218117  221183  217605    217654      1
62153  218127  221141  217796  221458  Write-in      1

[62154 rows x 6 columns]


In [23]:
# Get distinct candidates
candidates = set(aggregated_df['1st']).union(
    set(aggregated_df['2nd']),
    set(aggregated_df['3rd']),
    set(aggregated_df['4th']),
    set(aggregated_df['5th'])
)

# Remove 'Null' from candidates
candidates.discard('Null')

# Count the number of distinct candidates
num_distinct_candidates = len(candidates)

# Calculate the total number of votes
total_votes = aggregated_df['count'].sum()

print(f"Number of distinct candidates: {num_distinct_candidates}")
print(f"Total number of votes: {total_votes}")

Number of distinct candidates: 14
Total number of votes: 945026


In [24]:
ELA1stchoice = aggregated_df[aggregated_df['1st'] == '217572']['count'].sum()

# Print the count, converting the integer to a string
print("Number of Eric L. Adams first vote: " + str(ELA1stchoice))

Number of Eric L. Adams first vote: 289786


In [26]:
# Define the path to the folder in Google Drive
drive_folder_path = '/content/drive/My Drive/Summer Research 24/NycMayorVote'
csv_filename = f'{drive_folder_path}/aggregated_voting_NYCMayor.csv'

# Export the DataFrame to a CSV file in the specified folder
aggregated_df.to_csv(csv_filename, index=False)


# Code to download the CSV file in Google Colab
# from google.colab import files
# files.download(csv_filename)